In [1]:
pip install transformers


  Using cached urllib3-1.25.11-py2.py3-none-any.whl (127 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.19
    Uninstalling urllib3-1.26.19:
      Successfully uninstalled urllib3-1.26.19
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.20 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.20 requires nbformat==5.4.0, but you have nbformat 5.5.0 which is incompatible.
conda-repo-cli 1.0.20 requires python-dateutil==2.8.2, but you have python-dateutil 2.9.0.post0 which is incompatible.
conda-repo-cli 1.0.20 requires PyYAML==6.0, but you have pyyaml 6.0.2 which is incompatible.
conda-repo-cli 1.0.20 requires requests==2.28.1, but you have requests 2.23.0 which is incompatible.
anaconda-client 1.11.0 requires urllib3>=1.26.4, but you have urllib3 1.25.11 which is incompatible.


In [2]:
from transformers import pipeline

# Load the emotion detection pipeline
emotion_analyzer = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)

# Example reviews
reviews = [
    "I absolutely love this app! It's so intuitive and helpful.",
    "I'm so frustrated! It crashes every time I try to use it.",
    "This update is okay, but I expected more features.",
    "The customer support team is amazing and very responsive.",
    "I'm disappointed by the recent changes. They made the app worse."
]

# Analyze emotions
for review in reviews:
    emotions = emotion_analyzer(review)
    print(f"Review: {review}")
    print("Emotions:")
    for emotion in emotions[0]:
        print(f"  {emotion['label']}: {emotion['score']:.2f}")
    print()


C:\Users\Ryan Jewik\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\Ryan Jewik\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

C:\Users\Ryan Jewik\anaconda3\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Ryan Jewik\.cache\huggingface\hub\models--j-hartmann--emotion-english-distilroberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Review: I absolutely love this app! It's so intuitive and helpful.
Emotions:
  anger: 0.00
  disgust: 0.00
  fear: 0.00
  joy: 0.94
  neutral: 0.03
  sadness: 0.00
  surprise: 0.02

Review: I'm so frustrated! It crashes every time I try to use it.
Emotions:
  anger: 0.95
  disgust: 0.01
  fear: 0.00
  joy: 0.00
  neutral: 0.01
  sadness: 0.02
  surprise: 0.01

Review: This update is okay, but I expected more features.
Emotions:
  anger: 0.01
  disgust: 0.01
  fear: 0.01
  joy: 0.03
  neutral: 0.81
  sadness: 0.07
  surprise: 0.06

Review: The customer support team is amazing and very responsive.
Emotions:
  anger: 0.01
  disgust: 0.01
  fear: 0.01
  joy: 0.08
  neutral: 0.27
  sadness: 0.00
  surprise: 0.63

Review: I'm disappointed by the recent changes. They made the app worse.
Emotions:
  anger: 0.05
  disgust: 0.82
  fear: 0.00
  joy: 0.00
  neutral: 0.03
  sadness: 0.09
  surprise: 0.00



C:\Users\Ryan Jewik\anaconda3\lib\site-packages\transformers\pipelines\text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
